<img src="assets/routing.png" align="left" width="600" style="margin-right:15px;"/>

In [ ]:
from dotenv import load_dotenv, find_dotenv
from env_utils import doublecheck_env
from langchain_anthropic import ChatAnthropic

path = find_dotenv()
print("Loaded env from:", path)

# Load environment variables from .env
load_dotenv(find_dotenv())

# Check and print results
doublecheck_env(path)

llm = ChatAnthropic(model="claude-sonnet-4-5-20250929")

In [ ]:
from typing_extensions import Literal
from langchain.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image
from pydantic import BaseModel, Field
from typing_extensions import TypedDict

# Schema for structured output to use as routing logic
class Route(BaseModel):
    step: Literal["poem", "story", "joke"] = Field(
        None, description="The next step in the routing process"
    )


# Augment the LLM with schema for structured output
router = llm.with_structured_output(Route)


# State
class State(TypedDict):
    input: str
    decision: str
    output: str


# Nodes
def llm_call_1(state: State):
    """Write a story"""

    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_2(state: State):
    """Write a joke"""

    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_3(state: State):
    """Write a poem"""

    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_router(state: State):
    """Route the input to the appropriate node"""

    # Run the augmented LLM with structured output to serve as routing logic
    decision = router.invoke(
        [
            SystemMessage(
                content="Route the input to story, joke, or poem based on the user's request."
            ),
            HumanMessage(content=state["input"]),
        ]
    )

    return {"decision": decision.step}


# Conditional edge function to route to the appropriate node
def route_decision(state: State):
    # Return the node name you want to visit next
    if state["decision"] == "story":
        return "llm_call_1"
    elif state["decision"] == "joke":
        return "llm_call_2"
    elif state["decision"] == "poem":
        return "llm_call_3"


# Build workflow
router_builder = StateGraph(State)

# Add nodes
router_builder.add_node("llm_call_1", llm_call_1)
router_builder.add_node("llm_call_2", llm_call_2)
router_builder.add_node("llm_call_3", llm_call_3)
router_builder.add_node("llm_call_router", llm_call_router)

# Add edges to connect nodes
router_builder.add_edge(START, "llm_call_router")
router_builder.add_conditional_edges(
    "llm_call_router",
    route_decision,
    {  # Name returned by route_decision : Name of next node to visit
        "llm_call_1": "llm_call_1",
        "llm_call_2": "llm_call_2",
        "llm_call_3": "llm_call_3",
    },
)
router_builder.add_edge("llm_call_1", END)
router_builder.add_edge("llm_call_2", END)
router_builder.add_edge("llm_call_3", END)

# Compile workflow
router_workflow = router_builder.compile()

# Show the workflow
display(Image(router_workflow.get_graph().draw_mermaid_png()))

# Invoke
# state = router_workflow.invoke({"input": "Write me a joke about cats"})
# print(state["output"])

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Clear cell output
clear_output(wait=True)

# Create all widgets fresh
topic_input = widgets.Text(
    value='5G Core Network',
    placeholder='Enter network element...',
    description='Network:',
    continuous_update=False,
    layout=widgets.Layout(width='350px')
)

submit_btn = widgets.Button(
    description='Generate Intelligence Report',
    button_style='primary',
    icon='search'
)

result_html = widgets.HTML(value='')

def on_button_click(btn):
    """Handle button click"""
    network_element = topic_input.value.strip()
    
    if not network_element:
        result_html.value = "<p>⚠️ Please enter a network element!</p>"
        return
    
    # Show loading state
    btn.disabled = True
    result_html.value = f"<p>🔄 Analyzing network element: {network_element}...</p>"
    
    try:
        # Invoke workflow
        state = router_workflow.invoke({"network_element": network_element})
        
        # Build HTML string with results
        html = f"""
        <div style="font-family: 'Segoe UI', Arial, sans-serif; padding: 15px; background: #f8f9fa; border-radius: 5px;">
            <h3 style="color: #0066cc; margin-top: 0;">Network Intelligence Dashboard: {network_element}</h3>
            <hr style="border: 1px solid #0066cc;">
            
            <div style="margin: 20px 0;">
                <h4 style="color: #0066cc;">📊 Performance Report</h4>
                <pre style="white-space: pre-wrap; background: white; padding: 10px; border-left: 3px solid #28a745;">{state['performance_report']}</pre>
            </div>
            
            <div style="margin: 20px 0;">
                <h4 style="color: #0066cc;">🔒 Security Assessment</h4>
                <pre style="white-space: pre-wrap; background: white; padding: 10px; border-left: 3px solid #dc3545;">{state['security_assessment']}</pre>
            </div>
            
            <div style="margin: 20px 0;">
                <h4 style="color: #0066cc;">📈 Capacity Planning</h4>
                <pre style="white-space: pre-wrap; background: white; padding: 10px; border-left: 3px solid #ffc107;">{state['capacity_planning']}</pre>
            </div>
        </div>
        """
        result_html.value = html
        
    except Exception as e:
        result_html.value = f"<p>❌ Error: {str(e)}</p>"
    
    finally:
        btn.disabled = False

# Attach handler
submit_btn.on_click(on_button_click)

# Display
display(widgets.VBox([
    widgets.HTML("<h3>🌐 BT Network Intelligence Platform</h3>"),
    widgets.HTML("<p>Enter a network element (e.g., '5G Core Network', 'Fiber Access Network', 'Cloud Edge') and generate parallel intelligence reports</p>"),
    widgets.HBox([topic_input, submit_btn]),
    result_html
]))